In [1]:
import netgen.occ as occ

points = [( 0.25, 0,    0), ( 0.25, 0.50, 0), ( 0.50, 0.50, 0), ( 0.75, 0,    0),
          ( 0.75, 0.75, 0), ( 0.50, 0,    0), (-0.75, 0.75, 0), (-0.75, 0    ,0),
          (-0.25, 0    ,0), (-0.25, 0.5  ,0), (-0.50, 0.5  ,0), (-0.50, 0    ,0),
          ( 0.75,-0.025,0), (-0.75,-0.025,0), (-0.75,-0.275,0), ( 0.75,-0.275,0),
          (-1.25, 1.25 ,0), ( 1.25, 1.25 ,0), ( 1.25,-0.75 ,0), (-1.25,-0.75 ,0)]

edges = [[9,10],[10,11],[11,8],[8,9], [2,1],[1,0],[0,5],[5,2], [17,16],[16,19],[19,18],[18,17], [3,7],[7,6],[6,4],[4,3], [15,14],[14,13],[13,12],[12,15], [12,3],[7,13]]
faces = [[0,1,2,3],[4,5,6,7],[8,9,10,11],[12,13,14,15],[16,17,18,19],[18,20,13,21]]

e = {}
for i,j in enumerate(edges): e[i] = occ.Segment(points[j[0]],points[j[1]])

f = {}
for i,j in enumerate(faces): f[i] = occ.Face(occ.Wire([e[j[0]],e[j[1]],e[j[2]],e[j[3]]]))

geo = occ.Glue(list(f.values()))

geo.faces[0].name = 'coil_plus'; geo.faces[0].col = (1,0.5,0)
geo.faces[1].name = 'coil_minus'; geo.faces[1].col = (1,0.5,0)
geo.faces[2].name = 'air';   geo.faces[2].col = (0,0.5,1)
geo.faces[3].name = 'iron';  geo.faces[3].col = (0,0.5,0)
geo.faces[4].name = 'air';   geo.faces[4].col = (0,0.5,1)
geo.faces[5].name = 'iron';  geo.faces[5].col = (0,0.5,0)

geo.edges[8].name = 'outer'
geo.edges[9].name = 'outer'
geo.edges[10].name = 'outer'
geo.edges[11].name = 'outer'


h = 0.01*10
hh = 0.001*10

for i in range(50):
    geo.vertices[i].maxh = hh

geoOCC = occ.OCCGeometry(geo, dim = 2)
geoOCCmesh = geoOCC.GenerateMesh(maxh = h)


In [2]:
# import sys
# sys.path.insert(0,'../../') # adds parent directory
# import pde
# MESH = pde.mesh.netgen(geoOCCmesh)
# print(MESH)

In [8]:
import ngsolve as ng
from ngsolve.webgui import Draw
mesh = ng.Mesh(geoOCCmesh)
mesh.ne

2837

Current densiry : we have $66$ Amp-turns on a surface of 0.125 $m^2$, so $J = 66A/0.125 m^2 = 528 A/m^2$

In [9]:
# import sys
# sys.path.insert(0,'../../') # adds parent directory
# import pde

import numpy as np



order = 1
J = 8*66
s = 58*1e6

muAir = 1
muCoil = 1
muIron = 2500


# fes = ng.HCurl(mesh = mesh, order = order, dirichlet='outer')
fes = ng.H1(mesh = mesh, order = order, dirichlet='outer')
u,v = fes.TnT()

sigma = mesh.MaterialCF({"coil_plus": s, "coil_minus": s}, default=1)
mu = mesh.MaterialCF({"coil_plus": muCoil, "coil_minus": muCoil, "iron": muIron, "air":muAir}, default=1)
# Draw(sigma, mesh)

K = ng.BilinearForm(fes)
K += 1/mu*ng.grad(u)*ng.grad(v)*ng.dx
K.Assemble()

coil_plus  =  1/sigma * J * v * ng.dx(definedon = mesh.Materials("coil_plus"))
coil_minus = -1/sigma * J * v * ng.dx(definedon = mesh.Materials("coil_minus"))
f = ng.LinearForm(coil_plus + coil_minus).Assemble(); # fnp = f.vec.FV().NumPy()

gfu = ng.GridFunction(fes)
gfu.vec.data = K.mat.Inverse(fes.FreeDofs()) * f.vec
Draw(ng.grad(gfu), mesh)
Draw(gfu, mesh)



# ir = ng.IntegrationRule(points = [(0,0), (1,0), (0,1)], weights = [1/6, 1/6, 1/6] )
# a =  ng.SymbolicBFI (u*v, intrule=ir)


# import 

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [10]:
1/2*ng.Integrate(1/mu*ng.grad(gfu)*ng.grad(gfu), mesh, order = 2)

8.139438564009563e-12